In [1]:
from Notebook_Utilities import *

In [2]:
# Load and clean data
df = load_and_clean_data('0_CY2024_Waste_Streams_0.xlsx', '2024_Shipping')

# Fix Typo
df = df.replace("ReCYcling", "Recycling")

# Density (kg/gallon)
df['Density_kg_per_gal'] = df['Container_Mass'] / (df['Container_Size'] * df['Container_Qty'])

# Average Mass per Container (kg/container)
df['Avg_Mass_per_Container'] = df['Container_Mass'] / df['Container_Qty']

# Volume per Container (gallons/container)
df['Volume_per_Container'] = df['Container_Size']  # Since this is already per container

# Total Volume (gallons)
df['Total_Volume_gallons'] = df['Container_Size'] * df['Container_Qty']

# Mass-to-Volume Ratio (kg/gallon, total volume basis)
df['Mass_to_Volume_Ratio'] = df['Container_Mass'] / df['Total_Volume_gallons']

# Specific Mass Index (kg/container per gallon)
df['Specific_Mass_Index'] = df['Container_Mass'] / (df['Container_Qty'] * df['Container_Size'])

##### Map 

In [3]:
# Import BPO (ACT) Dataset
df1 = load_and_clean_data('0_CY2024_BPO.xlsx', 'Main')

# Ensure the column is treated as strings/text
df1['WIWPS_ID'] = df1['WIWPS_ID'].astype(str).str.strip()
df1['WIWPS_Name'] = df1['WIWPS_Name'].astype(str).str.strip()
# Ensure the column is treated as strings/text and remove ".0"
df1['TSDF_ID'] = df1['TSDF_ID'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)
df1['ACT_ID'] = df1['ACT_ID'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)
df1['DOT_Class'] = df1['DOT_Class'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)

# Add Column
df1['Container_Cost'] = np.nan

In [4]:
# Connecting WIWPS to Shipping Information
def map_and_add_columns_with_multiple_matches(
    df_1, df_2, column_to_map, mapping_column, columns_to_add, match_values=None, delimiter_pattern=r"[,\s]+"
):
    """
    Matches single or multiple entries in df_2[column_to_map] against multiple entries in df_1[mapping_column],
    and pulls specific columns from df_1 into df_2 while preserving df_2's rows.

    Parameters:
        df_1 (pd.DataFrame): The source DataFrame containing rows to pull columns from.
        df_2 (pd.DataFrame): The target DataFrame where new columns will be added.
        column_to_map (str): Column in df_2 to match against df_1[mapping_column].
        mapping_column (str): Column in df_1 containing single or multiple comma-separated entries.
        columns_to_add (list): List of column names from df_1 to add to df_2.
        match_values (list, optional): A list of values in df_2[column_to_map] to match.
        delimiter_pattern (str): Regex pattern to split multiple entries in df_1.

    Returns:
        pd.DataFrame: The updated df_2 with specified columns added.
    """
    # Preprocess df_1 to split mapping_column into multiple rows for matching
    expanded_df_1 = (
        df_1.assign(Expanded_ID=df_1[mapping_column].str.split(delimiter_pattern))
        .explode("Expanded_ID")
        .dropna(subset=["Expanded_ID"])
    )
    expanded_df_1["Expanded_ID"] = expanded_df_1["Expanded_ID"].str.strip()

    # Create a mapping dictionary for each column to add
    mapping_dicts = {
        col: expanded_df_1.set_index("Expanded_ID")[col].to_dict() for col in columns_to_add
    }

    # Filter df_2 based on match_values if provided
    if match_values:
        df_2 = df_2[df_2[column_to_map].isin(match_values)]

    # Add new columns to df_2
    for col in columns_to_add:
        df_2[col] = df_2[column_to_map].map(mapping_dicts[col]).fillna("NA")

    return df_2

In [5]:
# Columns to pull from df_1 and add to df_2
columns_to_add=["PO", "ACT_ID"]

# Apply Function
df = map_and_add_columns_with_multiple_matches(
    df1, # Historical DataFrame                  
    df, # New DataFrame
    column_to_map="WIWPS_ID", # Column in df_2 to match against df_1
    mapping_column="WIWPS_ID", # Dynamic creation of mapping dictionary
    columns_to_add=columns_to_add,
    delimiter_pattern=r"[,\s]+"
)

# Add Column
df['Container_Cost'] = np.nan

In [10]:
# Internal
df_WIWPS_ID = find_unique_text_and_counts_2(df, 'WIWPS_ID') # 
df_WIWPS_Name = find_unique_text_and_counts_1(df, 'WIWPS_Name') # No Delimiter
df_WIWPS_Name2 = find_unique_text_and_counts_2(df, 'WIWPS_Name') # 
df_Group = find_unique_text_and_counts_2(df, 'Group') 
df_Waste_Type = find_unique_text_and_counts_2(df, 'Waste_Type') 

# Shipping
df_TSDF_ID = find_unique_text_and_counts_2(df, 'TSDF_ID') # 
df_ACT_ID = find_unique_text_and_counts_2(df, 'ACT_ID') # 
df_Shipping_Name = find_unique_text_and_counts_1(df, 'Shipping_Name') # No Delimiter
df_Shipping_Name2 = find_unique_text_and_counts_2(df, 'Shipping_Name') # 
df_DOT_Name = find_unique_text_and_counts_1(df, 'DOT_Name') # 
df_DOT_Class = find_unique_text_and_counts_2(df, 'DOT_Class') # 
df_EPA_Waste_Code1 = find_unique_text_and_counts_1(df, 'EPA_Waste_Code') # 
df_EPA_Waste_Code2 = find_unique_text_and_counts_2(df, 'EPA_Waste_Code') # 
df_TSDF = find_unique_text_and_counts_2(df, 'TSDF') # 

# Containers
df_DOT_Name = find_unique_text_and_counts_2(df, 'Container_Size') # 
df_Container_Cost = find_unique_text_and_counts_2(df, 'Container_Cost') # 
df_Container_Qty = find_unique_text_and_counts_2(df, 'Container_Qty') # 
df_Container_Mass = find_unique_text_and_counts_2(df, 'Container_Mass') # 

print(' ',len(df_Group), ' Group')
print('',len(df_WIWPS_ID), ' WIWPS_ID')
print(' ',len(df_TSDF_ID), ' TSDF_ID')
print(' ',len(df_ACT_ID), ' ACT_ID')
print()
print('',len(df_WIWPS_Name), ' WIWPS_Name')
print('',len(df_Shipping_Name), ' Shipping_Name')
print(' ',len(df_DOT_Name),' DOT_Name')
print(' ',len(df_DOT_Class),' DOT_Class')
print(' ',len(df_EPA_Waste_Code),' EPA_Waste_Code @ find_unique_text_and_counts_1')
print('',len(df_EPA_Waste_Code2),' EPA_Waste_Code @ find_unique_text_and_counts_2')

  15  Group
 98  WIWPS_ID
  37  TSDF_ID
  24  ACT_ID

 45  WIWPS_Name
 82  Shipping_Name
  17  DOT_Name
  12  DOT_Class
  47  EPA_Waste_Code @ find_unique_text_and_counts_1
 28  EPA_Waste_Code @ find_unique_text_and_counts_2


In [11]:
df_TSDF

,Entry,Count
0,Veolia ES Technical Solutions LLC.,124
1,US Ecology Nevada Inc.,54
2,Veolia Environmental Services,38
3,Stericycle Inc.,5
4,South Central Solid Waste Authority,5
5,Advanced Chemical Treatment,4
6,W. Silver ReCYcling Inc.,4
7,150 Yard,1
8,Mesa Oil,1


##### EPA Waste Code Frequency

In [12]:
# Plot Results
plot_horizontal_bar(df_EPA_Waste_Code1,
                    'EPA Code Frequency', 
                    y_label='EPA Codes', 
                    figsize=(6,9), 
                    save_folder="TC_CY_2024", 
                    file_name="CY_2024_HW_Frequency_EPA_Codes.png",
                    display_plot=False)

TypeError: plot_horizontal_bar() got an unexpected keyword argument 'display_plot'

In [13]:
# Container Volume
results = find_unique_text_and_counts(df_hw, 'Container_Group')
# Plot Results
# Plot Results
plot_horizontal_bar(results,'Frequency of Container Volumes', 
                    y_label='V [Gal]', 
                    figsize=(9,6), 
                    save_folder="TC_CY_2024", 
                    file_name="CY_2024_HW_Frequency_Container_Group.png",
                    display_plot=False)

NameError: name 'find_unique_text_and_counts' is not defined

In [ ]:
df_Waste_Type

### BPO Grouping

In [ ]:
# Organize Dataframes by Type
df_ldr = df[df['Type'] == 'Hazardous (LDR)'] # LDR, US Ecology Nevada Inc.
df_lp = df[df['Type'] == 'Hazardous (Lab Pack)'] # Lab Pack, Veolia ES Technical Solutions.
df_wf = df[df['Type'] == 'Hazardous (Waste Fuel)'] # Waste Fuel, Veolia ES Technical Solutions
df_act = df[df['Type'] == 'Hazardous (ACT)'] # Miscellaneous
df_hw = pd.concat([df_ldr, df_lp, df_wf, df_act]) # Combine Hazardous Waste Data Frames
df_uw = df[df['Type'] == 'Universal'] # Universal Waste
df_tsca = df[df['Type'] == 'TSCA'] # Universal Waste

# Total Waste [kg]
qty_kg_sum_hw = df_hw['QTY_kg'].sum()
print(f"Total Exported Hazardous Waste: {qty_kg_sum_hw:.0f} [kg]")

# Total Waste [lbs]
hw_lbs = unit_conversion(qty_kg_sum_hw, 'kg', 'lbs') # kg to lb
print(f"Total Exported Hazardous Waste: {hw_lbs:.0f} [lbs]")

# HW Containers
N_Containers_hw = df_hw['Container_QTY'].sum()
print(f"Total Exported Containers: {N_Containers_hw:.0f}")
print()

# Universal Containers
N_Containers_uw = df_uw['Container_QTY'].sum()
print(f"Total Exported Containers: {N_Containers_hw:.0f}")
print()

# TSCA Containers
N_Containers_tsca = df_tsca['Container_QTY'].sum()
print(f"Total Exported Containers: {N_Containers_hw:.0f}")
print()

In [ ]:
# Show Sum of Quantities Shipped per Type
dfg = groupby_function(df_hw, 'Type', weight_column='Container_Mass', sort=True)
print('SUM =', dfg['Container_Mass'].sum(), '[kg]')
dfg

In [10]:
# Show Sum of Quantities Shipped per Type
dfg = groupby_function(df_hw, 'Type', weight_column='Container_Mass', sort=True)
print('SUM =', dfg['Container_Qty'].sum(), '[kg]')
dfg

NameError: name 'df_hw' is not defined

In [ ]:
# Plot Results
plot_horizontal_bar(df3_EPA_Waste_Code,
                    'df3_EPA_Waste_Code', 
                    y_label='EPA_Waste_Code', 
                    figsize=(12,4), 
                    save_folder=None, 
                    file_name=None,
                    display_plot=True)

In [ ]:
# Plot Results
plot_horizontal_bar(df3_EPA_Waste_Code2,
                    'df3_EPA_Waste_Code', 
                    y_label='EPA_Waste_Code', 
                    figsize=(12,4), 
                    save_folder=None, 
                    file_name=None,
                    display_plot=True)

In [ ]:
df_Waste_Type

### Monthly

In [ ]:
# Aggregate waste by month
monthly_totals = df_hw.groupby('Month')['QTY_kg'].sum()

# Ensure 'monthly_totals' is a DataFrame with proper structure
monthly_totals = monthly_totals.reset_index()

# Map numerical months to month names
month_labels = {1: 'January', 2: 'February', 3: 'March', 
                4: 'April', 5: 'May', 6: 'June',7: 
                'July', 8: 'August', 9: 'September', 10: 
                'October', 11: 'November', 12: 'December'}

monthly_totals['Month_Name'] = monthly_totals['Month'].map(month_labels)

# Dynamically scale the color intensity based on QTY_kg
norm = plt.Normalize(monthly_totals['QTY_kg'].min(), monthly_totals['QTY_kg'].max())
colors = sns.color_palette("coolwarm", as_cmap=True)(norm(monthly_totals['QTY_kg']))

# Plot the data without the color legend
plt.figure(figsize=(9, 6))
bars = plt.bar(monthly_totals['Month_Name'], monthly_totals['QTY_kg'], color=colors, edgecolor='black')

# Add labels and customize the plot aesthetics
plt.title('Monthly Hazardous Waste Totals', fontsize=18, pad=15)
plt.xlabel('Month', fontsize=14, labelpad=10)
plt.ylabel('Total Waste (kg)', fontsize=14, labelpad=15)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Add gridlines
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

output_folder = "TC_CY_2024" # Define the folder and file name
file_name = "CY_2024_HW_Monthly_Mass.png"
file_path = os.path.join(output_folder, file_name) # Construct the full file path
plt.savefig(file_path, dpi=600, bbox_inches='tight')

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
#volume_counts = df_hw.groupby(['Month', 'Container_Volume']).size().reset_index(name='Count')
#pivot_df = volume_counts.pivot(index='Month', columns='Container_Volume', values='Count').fillna(0)
#pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='tab20')

# Group by 'Month' and 'Container_Volume', then count occurrences
volume_counts = df_hw.groupby(['Month', 'Container_Volume']).size().reset_index(name='Count')

# Pivot data to prepare for plotting
pivot_df = volume_counts.pivot(index='Month', columns='Container_Volume', values='Count').fillna(0)

# Sort 'Container_Volume' (columns) in ascending order
ordered_columns = sorted(pivot_df.columns)  # Sort container volumes numerically
pivot_df = pivot_df[ordered_columns]

# Normalize Container_Volume for Color Mapping
norm = Normalize(vmin=min(ordered_columns), vmax=max(ordered_columns))  # Normalize volumes
colors = [cm.coolwarm(norm(vol)) for vol in ordered_columns]  # Map each volume to a color

# Plot Stacked Bar Chart with Custom Colors
fig, ax = plt.subplots(figsize=(10, 6))
pivot_df.plot(kind='bar', stacked=True, color=colors, edgecolor='black', ax=ax)

# Customize Plot
plt.title('Monthly Count of Container Volumes', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Containers', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Container Volume', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the Plot
output_folder = "TC_CY_2024"
file_name = "CY_2024_HW_Monthly_Container_Volumes.png"
os.makedirs(output_folder, exist_ok=True)
file_path = os.path.join(output_folder, file_name)
plt.savefig(file_path, dpi=600, bbox_inches='tight')
print(f"Plot saved to: {file_path}")

# Show the Plot
plt.show()